In [56]:
import pandas as pd
import numpy as np

# Read an Excel file
df = pd.read_excel("20251205 Outgoing Incoming Payments.xlsx")
df2 = pd.read_excel("20251202 Sales + Payment aug-nov 2025.xlsx")

In [57]:
df['Selisih'] = (df['DocDate'] - df['CreateDate']).dt.days
df['Month'] = df['DocDate'].dt.month_name()
df['Year'] = df['DocDate'].dt.year
df['Poin'] = df['Selisih'].apply(lambda x: 0 if x < -1 else 1)

In [58]:
today = pd.Timestamp.today().normalize()   # keeps only the date part
df2 = df2[~df2['Payment_Terms_Code'].isin(['0 Days', 'COD'])].drop_duplicates()
df2 = df2[['Document Number', 'Document Status', 'Posting Date', 'Status_payment', 'New_Due_Date', 'Reconciliation_Date']]
df2 = df2.sort_values('Document Number').reset_index(drop=True)
df2['Month'] = df2['Posting Date'].dt.month_name()
df2['Year'] = df2['Posting Date'].dt.year
df2['JT'] = np.where(df2['Status_payment'] == "Belum Bayar", today - df2['New_Due_Date'], df2['Reconciliation_Date'] - df2['New_Due_Date'])
df2['Poin'] = np.where((df2['Status_payment'].isin(['Late', 'Belum Bayar'])) & (df2['JT'].dt.days > 14), 0, 1)


In [59]:
import gspread
from gspread_dataframe import set_with_dataframe

# ---- Google Sheets Setup ----
SERVICE_ACCOUNT_FILE = "projectkpidashboard.json"
SPREADSHEET_ID = "1bxVq20N1G9UIyek6BVMgpEMOvQJib-j7Jbsfj82KZtE"
SHEET_NAME = "Database"
SHEET_NAME2 = "Database2"

# Authorize
gc = gspread.service_account(filename=SERVICE_ACCOUNT_FILE)

# Open the Google Sheet
sh = gc.open_by_key(SPREADSHEET_ID)

# Select the worksheet
worksheet = sh.worksheet(SHEET_NAME)
worksheet2 = sh.worksheet(SHEET_NAME2)

# Optional: clear sheet before writing
worksheet.clear()
worksheet2.clear()

# Write DataFrame to Sheet
set_with_dataframe(worksheet, df)
set_with_dataframe(worksheet2, df2)

print("Data uploaded successfully!")


Data uploaded successfully!
